In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(12,8)

In [2]:
def replace_string(data, column):
    k_columns = data[data[column].str.contains("K")]
    k_columns = k_columns.loc[k_columns[column]!='K']
    k_columns[column] = k_columns[column].str.strip('K').astype('float') * 1000
    m_columns = data[data[column].str.contains("M")]
    m_columns = m_columns.loc[m_columns[column]!='M']
    m_columns[column] = m_columns[column].str.strip('M').astype('float') * 1e6
    b_columns = data[data[column].str.contains("B")]
    b_columns = b_columns.loc[b_columns[column]!='B']
    b_columns[column] = b_columns[column].str.strip('B').astype('float') * 1e9
    columns = pd.concat([k_columns, m_columns, b_columns], axis=0)
    
    columns = columns.sort_index()
    
    return columns

In [3]:
%%time
storm_events = []
#init_path = '/Users/allanbellahsene/Desktop/THESIS/DATA/NCFEI/StormEvents_' #Mac_OS path
init_path = 'C:\\Users\\abellahsene\\Desktop\\THESIS\\DATA\\NCFEI\\StormEvents_' #Windows path
for years in list(np.arange(1962,2020)):
    path = init_path + str(years) + '.csv'
    storm_data = pd.read_csv(path, usecols=['BEGIN_DAY', 'MONTH_NAME', 'YEAR', 'STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EVENT_TYPE', 'EPISODE_ID', 'EVENT_ID', 'DEATHS_DIRECT', 'DEATHS_INDIRECT'])
    storm_data.index = pd.to_datetime(storm_data['BEGIN_DAY'].astype('str')+ '-' + storm_data['MONTH_NAME'] + '-' + storm_data.YEAR.astype('str'))
    storm_data = storm_data[['STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'EVENT_TYPE', 'EVENT_ID', 'EPISODE_ID']] #keep only relevant columns
    storm_data.dropna(inplace=True) #drop missing values
    storm_data.sort_index(inplace=True) #sort data by date
    storm_events.append(storm_data)
    #print(years)
STORMS = pd.concat([storm_events[0], storm_events[1]], axis=0)
for i in range(2, len(storm_events)):
    STORMS = pd.concat([STORMS, storm_events[i]], axis=0)
print(STORMS.shape)
df = STORMS
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].astype('str')
df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].astype('str')
STORMS = df
STORMS = STORMS.loc[((STORMS.DAMAGE_PROPERTY!='0') & (STORMS.DAMAGE_CROPS =='0')) | ((STORMS.DAMAGE_PROPERTY=='0') & (STORMS.DAMAGE_CROPS !='0')) | ((STORMS.DAMAGE_PROPERTY!='0') & (STORMS.DAMAGE_CROPS !='0'))]
STORMS = STORMS.loc[STORMS.DAMAGE_PROPERTY!='K'] #delete rows that only contain "K"
STORMS = STORMS.loc[STORMS.DAMAGE_CROPS!='K']
STORMS = replace_string(STORMS, 'DAMAGE_PROPERTY')
STORMS = replace_string(STORMS, 'DAMAGE_CROPS')
STORMS['TOTAL_DAMAGES'] = STORMS['DAMAGE_PROPERTY'] + STORMS['DAMAGE_CROPS']
STORMS['Date'] = STORMS.index
STORMS = STORMS[['Date','STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'TOTAL_DAMAGES', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'EVENT_TYPE', 'EVENT_ID', 'EPISODE_ID']]
df = STORMS.groupby(['STATE', 'Date']).agg({'TOTAL_DAMAGES':sum})
STORMS = STORMS.loc[STORMS['TOTAL_DAMAGES']!=0]
States = list(STORMS.STATE.unique())

(708372, 8)
Wall time: 18.1 s


In [4]:
v_cons = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'Medium',
           'High Wind': 'High',
           'Flash Flood': 'Medium',
           'Flood': 'High',
           'Cold/Wind Chill': 'Medium',
           'Ice Storm': 'High',
           'Hail': 'High',
           'Lightning': 'High',
           'Drought': 'Medium',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'High',
           'Wildfire': 'High',
           'Heavy Rain': 'High',
           'Heavy Snow': 'High',
           'Blizzard': 'High',
           'Frost/Freeze': 'High',
           'Winter Weather': 'Medium',
           'Dense Fog': 'Medium',
           'Strong Wind': 'High',
           'Tropical Storm': 'High',
           'Dust Storm': 'Medium',
           'Extreme Cold/Wind Chill': 'Medium',
           'Debris Flow': 'High',
           'Coastal Flood': 'High',
           'Heat': 'Medium',
           'Marine Thunderstorm Wind': 'Low',
           'Landslide': 'High',
           'Lake-Effect Snow': 'Medium',
           'High Surf': 'Medium',
           'Lakeshore Flood': 'Low',
           'Tsunami': 'High',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Low',
           'Avalanche': 'Medium',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Medium',
           'Dust Devil': 'Medium',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Medium',
           'Rip Current': 'Low',
           'Sleet': 'Medium',
           'Marine Hurricane/Typhoon': 'Low',
           'Marine Lightning': 'Medium',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}
v_agr = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'Low',
           'High Wind': 'High',
           'Flash Flood': 'High',
           'Flood': 'High',
           'Cold/Wind Chill': 'High',
           'Ice Storm': 'High',
           'Hail': 'High',
           'Lightning': 'Medium',
           'Drought': 'High',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'Medium',
           'Wildfire': 'High',
           'Heavy Rain': 'High',
           'Heavy Snow': 'High',
           'Blizzard': 'Medium',
           'Frost/Freeze': 'Medium',
           'Winter Weather': 'Low',
           'Dense Fog': 'Low',
           'Strong Wind': 'High',
           'Tropical Storm': 'High',
           'Dust Storm': 'Medium',
           'Extreme Cold/Wind Chill': 'Medium',
           'Debris Flow': 'Medium',
           'Coastal Flood': 'Medium',
           'Heat': 'High',
           'Marine Thunderstorm Wind': 'Medium',
           'Landslide': 'Medium',
           'Lake-Effect Snow': 'Medium',
           'High Surf': 'Medium',
           'Lakeshore Flood': 'Low',
           'Tsunami': 'Medium',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Low',
           'Avalanche': 'Low',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Low',
           'Dust Devil': 'Low',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Low',
           'Rip Current': 'Low',
           'Sleet': 'Low',
           'Hurricane (Typhoon)': 'High',
           'Marine Hurrican/Typhoon': 'Low',
           'Marine Lightning': 'Low',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}


v_energy = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'High',
           'High Wind': 'High',
           'Flash Flood': 'High',
           'Flood': 'High',
           'Cold/Wind Chill': 'Medium',
           'Ice Storm': 'High',
           'Hail': 'Medium',
           'Lightning': 'High',
           'Drought': 'Low',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'High',
           'Wildfire': 'Low',
           'Heavy Rain': 'Medium',
           'Heavy Snow': 'Medium',
           'Blizzard': 'High',
           'Frost/Freeze': 'Medium',
           'Winter Weather': 'Low',
           'Dense Fog': 'Low',
           'Strong Wind': 'Medium',
           'Tropical Storm': 'Medium',
           'Dust Storm': 'Low',
           'Extreme Cold/Wind Chill': 'Low',
           'Debris Flow': 'Medium',
           'Coastal Flood': 'Medium',
           'Heat': 'Low',
           'Marine Thunderstorm Wind': 'Medium',
           'Landslide': 'Low',
           'Lake-Effect Snow': 'Low',
           'High Surf': 'Low',
           'Lakeshore Flood': 'Medium',
           'Tsunami': 'High',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Medium',
           'Avalanche': 'High',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Low',
           'Dust Devil': 'Low',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Medium',
           'Rip Current': 'Low',
           'Sleet': 'Low',
           'Marine Hurricane/Typhoon': 'Medium',
           'Marine Lightning': 'Low',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}


In [5]:
STORMS['Vulnerability Energy'] = STORMS['EVENT_TYPE'].map(v_energy)
STORMS['Vulnerability Agriculture'] = STORMS['EVENT_TYPE'].map(v_agr)
STORMS['Vulnerability Construction'] = STORMS['EVENT_TYPE'].map(v_cons)

In [6]:
def climate_score(climate_data, start_year, end_year, state, sector):
    df = climate_data.copy()
    df = df.loc[(df.index>=start_year+'-01-01') & (df.index<=end_year+'-12-31') & (df.STATE==state)]
    df['Vulnerability'] = df['Vulnerability '+sector].map({'High': 1, 'Medium': 0.5, 'Low': 0})
    
    score = np.dot(np.array(df.TOTAL_DAMAGES).T, np.array(df.Vulnerability))
    
    return score

def climate_ranking(climate_data, start_year, end_year, sector):
    score = []
    for states in list(climate_data.STATE.unique()):
        score.extend((states, climate_score(climate_data, start_year, end_year, states, sector)))
    score = np.array(score).reshape((int(len(score)/2), 2))
    ranking = pd.DataFrame(score, columns=['State', 'Climate Score Risk'])
    ranking['Climate Score Risk'] = ranking['Climate Score Risk'].astype('float')
    ranking.sort_values(by='Climate Score Risk', inplace=True, ascending=False)
    ranking.reset_index(inplace=True)
    ranking.drop(columns=['index'], inplace=True)
    
    return ranking

In [7]:
#Financial fundamental data

#fundamentals = pd.read_csv('/Users/allanbellahsene/Desktop/THESIS/DATA/WHARTON_DATABASE/Funda_data.csv')
path = "C:\\Users\\abellahsene\\Desktop\\THESIS\\DATA\\WHARTON_DATABASE\\Funda_data.csv"
fundamentals = pd.read_csv(path)
fundamentals[['Date']] = fundamentals[['datadate']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[4:6],s[6:], s[0:4]))
fundamentals.Date = pd.to_datetime(fundamentals.Date)
fin_data = fundamentals
#sector = '11'
#data = fin_data.loc[fin_data.naics.astype('str').str.startswith(sector)]
data = fin_data
data = data[['gvkey', 'fqtr', 'tic', 'conm', 'atq', 'niq', 'loc', 'state', 'naics', 'Date']]
data.index = data.Date
data = data.loc[data['loc'] == 'USA'] #keep only companies with headquarters in the US
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))
data['State'] = data['state'].map(abbrev_us_state)
data.State = data.State.str.upper()
data['ROA'] = data['niq'] / data['atq']
data = data[['gvkey', 'naics','tic', 'conm', 'ROA','State']]

In [8]:
data.dropna(inplace=True)
data = data.loc[abs(data.ROA)<10]

In [9]:
def financial_ranking(fin_data, start_year, end_year, sector):
    data = fin_data.loc[(fin_data.index>=start_year+'-01-01') & (fin_data.index<=end_year+'-12-31') & (fin_data.naics.astype('str').str.startswith(sector))]
    
    states = data.State.unique()
    av_roa = []
    for state in states:
        companies = data.conm.loc[data.State==state].unique()
        for comp in companies:
            ROA = data.ROA.loc[data.conm==comp].mean()
            av_roa.extend((state, ROA, comp))

    av_roa = np.array(av_roa).reshape(int(len(av_roa)/3), 3)
    av_roa = pd.DataFrame(av_roa, columns=['State', 'Average ROA', 'Company'])
    av_roa['Average ROA'] = av_roa['Average ROA'].astype('float')
    av_roa = pd.DataFrame(av_roa.groupby('State')['Average ROA'].mean())
    av_roa.sort_values('Average ROA', inplace=True)
    
    return av_roa

In [10]:
financial_ranking(fin_data=data, start_year='1996', end_year='2019', sector='11')

,Average ROA
State,
FLORIDA,-1.005070
TEXAS,-0.573137
SOUTH CAROLINA,-0.382448
CALIFORNIA,-0.182216
ARIZONA,-0.177113
OREGON,-0.095789
INDIANA,-0.080167
HAWAII,-0.036784
NEW JERSEY,-0.032771


In [11]:
climate_ranking(climate_data=STORMS, start_year='1996', end_year='2019', sector='Energy').head(50)

,State,Climate Score Risk
0,FLORIDA,3.209721e+10
1,PUERTO RICO,2.144407e+10
2,NEW JERSEY,1.575889e+10
3,MISSISSIPPI,1.080490e+10
4,NORTH CAROLINA,1.036679e+10
5,ALABAMA,9.100880e+09
6,GEORGIA,6.300965e+09
7,OKLAHOMA,4.175868e+09
8,MICHIGAN,4.007119e+09
9,OHIO,3.395479e+09
